In [1]:
from pyscope.observatory import Observatory
from pyscope.telrun import mk_mosaic_schedule, schedtel
from astropy import coordinates as coord
import logging
import time
import numpy as np
import pathlib
from IPython.display import display, clear_output

# Set up logging
logging.basicConfig(level=logging.DEBUG)

# Set up after cell logging at debug level
logger = logging.getLogger('pyscope')
logger.setLevel(logging.DEBUG)

In [2]:
logger.setLevel(logging.INFO)

In [ ]:
blocks = mk_mosaic_schedule('NGC2174', ('g','r'), [30,30], ('Philip', 'XPG'), overlap=5, grid_size=(2,2), n_exp=5, readout=2)

Get coordinates from mosaic tool on [Telescopius](https://www.telescopius.com), copy as CSV and paste into cell below.

In [8]:
NGC2174_coordinates = '''Pane, RA, DEC, Position Angle (East), Pane width (arcmins), Pane height (arcmins), Overlap, Row, Column
Center, 06hr 09' 49", 20º 34' 25", 4.00, 27.00, 27.00, 18%, -, -
Pane 1, 06hr 10' 39", 20º 44' 41", 4.00, 27.00, 27.00, 18%, 1, 1
Pane 2, 06hr 09' 05", 20º 46' 13", 4.00, 27.00, 27.00, 18%, 1, 2
Pane 3, 06hr 10' 33", 20º 22' 35", 4.00, 27.00, 27.00, 18%, 2, 1
Pane 4, 06hr 08' 58", 20º 24' 08", 4.00, 27.00, 27.00, 18%, 2, 2
'''.replace('º', 'deg')

# Load csv from string into table
from astropy.table import Table
NGC2174_table = Table.read(NGC2174_coordinates, format='ascii.csv')
#print(NGC2174_table)
# Remove center row
NGC2174_table.remove_row(0)
#print(NGC2174_table)
# Convert RA and DEC to skycoords
from astropy.coordinates import SkyCoord
from astropy import units as u
NGC2174_table['SkyCoord'] = SkyCoord(NGC2174_table['RA'], NGC2174_table['DEC'], unit=(u.hourangle, u.deg))
# Print all columns of table
NGC2174_table.pprint(max_lines=-1, max_width=-1)

 Pane       RA           DEC      Position Angle (East) Pane width (arcmins) Pane height (arcmins) Overlap Row Column               SkyCoord             
                                                                                                                                    deg,deg              
------ ------------ ------------- --------------------- -------------------- --------------------- ------- --- ------ -----------------------------------
Pane 1 06hr 10' 39" 20deg 44' 41"                   4.0                 27.0                  27.0     18%   1      1          92.6625,20.744722222222222
Pane 2 06hr 09' 05" 20deg 46' 13"                   4.0                 27.0                  27.0     18%   1      2 92.27083333333333,20.77027777777778
Pane 3 06hr 10' 33" 20deg 22' 35"                   4.0                 27.0                  27.0     18%   2      1 92.63749999999999,20.37638888888889
Pane 4 06hr 08' 58" 20deg 24' 08"                   4.0                 27.0

In [3]:
rlmt = Observatory(config_path="./rlmt.cfg")
rlmt.connect_all()

INFO:pyscope.observatory.observatory:Using config file to initialize observatory: ./rlmt.cfg
INFO:pyscope.observatory.observatory:Using MaxIm DL as the camera driver
INFO:pyscope.observatory.observatory:Using MaxIm DL as the filter wheel driver
INFO:pyscope.observatory.observatory:Using MaxIm DL as the WCS driver
INFO:pyscope.observatory.observatory:CCD Temp Set to -65.63
INFO:pyscope.observatory.observatory:Checking passed kwargs and overriding config file values
INFO:pyscope.observatory.observatory:CCD Temp Set to -65.63
INFO:pyscope.observatory.observatory:Camera connected
INFO:pyscope.observatory.observatory:CCD Temp Set to -65.63
INFO:pyscope.observatory.observatory:Filter wheel connected
INFO:pyscope.observatory.observatory:Focuser connected
INFO:pyscope.observatory.observatory:Observing conditions connected
INFO:pyscope.observatory.observatory:Telescope connected


Turning cooler on


True

In [12]:
# Start autofocus
rlmt.focuser.Autofocus()

True

# Center on object (run both cells below)

In [4]:
save_dir = "philip_2024-04-17"
# Make the save directory if it doesn't exist
pathlib.Path(save_dir).mkdir(parents=True, exist_ok=True)

def capture_grism_image(filter_name, object_name, exp_time=5, filename_prefix="focus", binning=1):
    # # Set binning
    rlmt.camera.BinX = binning
    rlmt.camera.BinY = binning

    # # Start exposure
    print(f"Starting {int(exp_time)}s exposure...")
    rlmt.camera.StartExposure(int(exp_time), True)
    
    # Set up image base filename
    exp_time_str = str(exp_time).replace(".", "p")
    filename_base = f"{filename_prefix}_{object_name}_{exp_time_str}s_{filter_name}"
    # Remove spaces from filename
    filename_base = filename_base.replace(" ", "_")
    # Check if file exists in save_dir
    i = 0
    filename = pathlib.Path(save_dir) / f"{filename_base}_{i}.fts"
    # print(f"Checking if {filename} exists...")
    # print(f"filename.exists() is {filename.exists()}")
    while filename.exists():
        # If file exists, add a number to the end of the filename
        i += 1
        # print(f"{filename} does exist")
        filename = pathlib.Path(save_dir) / f"{filename_base}_{i}.fts"
        # print(f"Checking if {filename} exists...")

    # Save latest image
    while not rlmt.camera.ImageReady:
        time.sleep(0.1)
    time.sleep(0.1)
    rlmt.save_last_image(filename, "Light", overwrite=True)

    return filename

below sets object

In [11]:
# mosaic_panel = 1
# object_name = f"NGC2174_{mosaic_panel}"
# src = NGC2174_table['SkyCoord'][mosaic_panel]
object_name = "NGC 6045" # Change name here
# src = coord.SkyCoord("19h21m09.2835s -03d44m26.2962s", frame='icrs')
src = coord.SkyCoord.from_name(object_name)
print(f"RA: {src.ra.hms},\nDec: {src.dec.dms}")
print(src)
# Print alt az of source
src_altaz = rlmt.get_object_altaz(src)
print(f"Elevation: {src_altaz.alt.deg:.2f}, Azimuth: {src_altaz.az.deg:.2f}")
print(f"Elevation dms: {src_altaz.alt.dms}, \nAzimuth dms: {src_altaz.az.dms}")
if src_altaz.alt.deg < 30:
    raise Exception("Source is too low to observe.")
 
# tete_coords = src.transform_to(coord.TETE(obstime = rlmt.observatory_time, location=rlmt.observatory_location))
# print(tete_coords)
# print(f"RA: {tete_coords.ra.hms},\nDec: {tete_coords.dec.dms}")

RA: hms_tuple(h=16.0, m=5.0, s=7.8833904000049415),
Dec: dms_tuple(d=17.0, m=45.0, s=27.69706799999483)
<SkyCoord (ICRS): (ra, dec) in deg
    (241.28284746, 17.75769363)>
Elevation: 40.96, Azimuth: 93.60
Elevation dms: dms_tuple(d=40.0, m=57.0, s=46.34260427665879), 
Azimuth dms: dms_tuple(d=93.0, m=35.0, s=45.58044977581517)


below slews to and centers object

In [12]:
# Set to g filter
rlmt.filter_wheel.Position = 3
time.sleep(3)
while rlmt.focuser.IsMoving:
    time.sleep(0.5)

# Run recentering algorithm
if src_altaz.alt.deg < 30:
    raise Exception("Source is too low to observe.")
rlmt.recenter(src, 
    target_x_pixel=1024, # TODO: make default center of sensor in each axis
    target_y_pixel=1024,
    exposure=3, 
    save_images=True,
    save_path="./recenter_images/",
    readout=2,
)

INFO:pyscope.observatory.observatory:Attempt 1 of 5
INFO:pyscope.observatory.observatory:Slewing to RA hms_tuple(h=16.0, m=5.0, s=7.8833904000049415) and Dec dms_tuple(d=17.0, m=45.0, s=27.69706799999483)
INFO:pyscope.observatory.observatory:Turning on sidereal tracking...
INFO:pyscope.observatory.observatory:Sidereal tracking is on.
INFO:pyscope.observatory.observatory:Attempting to slew to coordinates...
INFO:pyscope.observatory.observatory:Slewing to RA 16.10401 and Dec 17.68826
INFO:pyscope.observatory.observatory:Settling for 3.00 seconds...
INFO:pyscope.observatory.observatory:Settling for 3.00 seconds
INFO:pyscope.observatory.observatory:Taking 3.00 second exposure
INFO:pyscope.observatory.observatory:Exposure complete
INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image
INF

True

Below cell runs a series of images (and saves them)

In [15]:

# filter_positions = {
#     "LowRes": 0,
#     "HighRes": 4,
#     "g": 2,
#     "r": 3,
#     "ha": 10,
#     "oiii": 7,
#     "sii": 9,
# }
# filter_name = "ha"
# rlmt.filter_wheel.Position = filter_positions[filter_name]
# exp_time = 60 # in seconds
# num_images = 5

# for i in range(num_images):
#     capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")

# filter_name = "sii"
# rlmt.filter_wheel.Position = filter_positions[filter_name]
# exp_time = 60 # in seconds
# num_images = 5

# for i in range(num_images):
#     capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")

# filter_name = "oiii"
# rlmt.filter_wheel.Position = filter_positions[filter_name]
# exp_time = 60 # in seconds
# num_images = 5

# for i in range(num_images):
#     capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")

Starting 60s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 60s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 60s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 60s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 60s exposure...


KeyboardInterrupt: 

In [10]:
# filter_positions = {
#     "LowRes": 0,
#     "HighRes": 4,
#     "g": 2,
#     "r": 3,
#     "i": 5,
#     "ha": 10,
#     "oiii": 7,
#     "sii": 9,
# }
# num_images = 15
# for i in range(num_images):
#     filter_name = "oiii"
#     rlmt.filter_wheel.Position = filter_positions[filter_name]
#     exp_time = 120 # in seconds
#     capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")


Starting 120s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 120s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 120s exposure...


...

INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 120s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 120s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


# Alex, run below to take images

In [14]:
filter_positions = {
    "LowRes": 0,
    "HighRes": 4,
    "g": 2,
    "r": 3,
    "i": 5,
    "ha": 10,
    "oiii": 7,
    "sii": 9,
}

num_images = 200

# # Start autofocus
# rlmt.focuser.Autofocus()

for i in range(num_images):
    filter_name = "r"
    rlmt.filter_wheel.Position = filter_positions[filter_name]
    exp_time = 30 # in seconds
    capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")

    filter_name = "g"
    rlmt.filter_wheel.Position = filter_positions[filter_name]
    exp_time = 30 # in seconds
    capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")

    filter_name = "i"
    rlmt.filter_wheel.Position = filter_positions[filter_name]
    exp_time = 30 # in seconds
    capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")

    if i % 2 == 0:
        filter_name = "ha"
        rlmt.filter_wheel.Position = filter_positions[filter_name]
        exp_time = 120
        capture_grism_image(filter_name, object_name, exp_time=exp_time, filename_prefix="tricolor")


Starting 30s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 30s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 30s exposure...


...

INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 30s exposure...


INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm image


Starting 30s exposure...


com_error: (-2147352567, 'Exception occurred.', (65535, 'MaxIm DL 6', 'Exposure Aborted', None, 0, 0), None)

# Setup for Focus position movement

### Change values in cell below

In [5]:
# Image save directory
save_dir = "focus_images_weather_test"
# Make the save directory if it doesn't exist
pathlib.Path(save_dir).mkdir(parents=True, exist_ok=True)

# Set up focuser position list
# Start at min, end at max, step by focus_step
min_focus = 3000
max_focus = 3250
focus_step = 250

# Filter positions DO NOT CHANGE
filter_positions = {
    "LowRes": 0,
    "HighRes": 4,
    "g": 2,
}

# Camera and filter wheel settings (use filter names from above)
num_images = 1 # Images at each focus position
exp_time = 0.1
filter_name = "HighRes" # This is the filter that will be used

# Move filter wheel to position
rlmt.filter_wheel.Position = filter_positions[filter_name]

# Print for verification
focus_positions = np.arange(min_focus, max_focus, focus_step)
print(f"Focus positions are {focus_positions}, there are {len(focus_positions)} positions")
print(f"Filter wheel is at (or moving to) position {rlmt.filter_wheel.Position}")

DEBUG:pyscope.observatory.observatory:Observatory.filter_wheel property called
DEBUG:pyscope.observatory.maxim:Position setter called with value=4
DEBUG:pyscope.observatory.observatory:Observatory.filter_wheel property called
DEBUG:pyscope.observatory.maxim:_MaximFilterWheelPosition called


Focus positions are [3000], there are 1 positions
Filter wheel is at (or moving to) position 4


In [4]:
def capture_focus_image(focus_position, filter_name, exp_time=5, filename_prefix="focus", binning=1):
    # # Set binning
    rlmt.camera.BinX = binning
    rlmt.camera.BinY = binning

    # # Start exposure
    print(f"Starting {int(exp_time)}s exposure...")
    rlmt.camera.StartExposure(int(exp_time), True)
    
    # Set up image base filename
    exp_time_str = str(exp_time).replace(".", "p")
    filename_base = f"{filename_prefix}_{focus_position}_{exp_time_str}s_{filter_name}"
    # Check if file exists in save_dir
    i = 0
    filename = pathlib.Path(save_dir) / f"{filename_base}_{i}.fts"
    # print(f"Checking if {filename} exists...")
    # print(f"filename.exists() is {filename.exists()}")
    while filename.exists():
        # If file exists, add a number to the end of the filename
        i += 1
        # print(f"{filename} does exist")
        filename = pathlib.Path(save_dir) / f"{filename_base}_{i}.fts"
        # print(f"Checking if {filename} exists...")

    # Save latest image
    while not rlmt.camera.ImageReady:
        time.sleep(0.1)
    time.sleep(0.1)
    rlmt.save_last_image(filename, "Light", overwrite=True)

    return filename

In [7]:
# Focus at each position
focus_move_time = 15
img_download_time = 6
focus_image_time = exp_time + img_download_time

# Estimate total time required
total_time = (
    focus_move_time * len(focus_positions)
) + focus_image_time * num_images * len(focus_positions)
print(f"Estimated total time required is {total_time/60} minutes")

time_per_cycle = []

for focus_position in focus_positions:
    if time_per_cycle:
        print(f"Average time per cycle is {np.mean(time_per_cycle):.2f} seconds")
        est_time_remaining = np.mean(time_per_cycle) * (
            len(focus_positions) - np.where(focus_positions == focus_position)[0][0]
        )
        # Print time remaining in minutes and seconds
        print(
            f"Estimated time remaining is {est_time_remaining//60:.0f} min {est_time_remaining%60:.0f} sec"
        )
    start_time = time.time()
    print(f"Moving to focus position {focus_position}")

    print(
        f"This is position {np.where(focus_positions == focus_position)[0][0]+1} of {len(focus_positions)}"
    )
    rlmt.focuser.Move(focus_position)
    while rlmt.focuser.IsMoving:
        #print(f"Focuser is moving to {focus_positions[0]}")
        time.sleep(0.1)
    #current_focus = int(rlmt.focuser.Position)
    current_focus = focus_position
    print(f"Focus position is {current_focus}")
    print("========================================")

    current_exp_time = exp_time
    print(f"current_exp_time={current_exp_time}")

    for i in range(num_images):
        img_file = capture_focus_image(current_focus, filter_name, exp_time=current_exp_time, filename_prefix="focus", binning=1)
        print(f"Saved image {i} at focus position {current_focus}")

    end_time = time.time()
    time_per_cycle.append(end_time - start_time)

    # Clear cell output for next loop
    clear_output()